* Извлекать реплики с приветствием – где менеджер поздоровался.  
* Извлекать реплики, где менеджер представил себя.  
* Извлекать имя менеджера.  
* Извлекать название компании.  
* Извлекать реплики, где менеджер попрощался.  
* Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»  

In [1]:
import pandas as pd
from yargy import rule, Parser, or_
from yargy.predicates import eq, type as ttype, dictionary
from yargy.pipelines import caseless_pipeline
from yargy.interpretation import fact

# Data

In [2]:
data = pd.read_csv('test_data.csv')

In [3]:
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


# Files

In [4]:
from random import seed, sample

names = []
with open('names.txt', encoding="utf-8") as file:
    for line in file:
        name = line.rstrip()
        names.append(name)


seed(1)
sample(names, 10)

['Диля',
 'Евсей',
 'Ханс',
 'Белла',
 'Марта',
 'Гортензия',
 'Афанасий',
 'Фредерик',
 'Авдей',
 'Алон']

In [5]:
greetings = []
with open('greetings.txt', encoding="utf-8") as file:
    for line in file:
        greeting = line.rstrip()
        greetings.append(greeting)
greetings

['добрый день',
 'здравствуйте',
 'доброе утро',
 'добрый вечер',
 'приветсвую',
 'привет']

In [6]:
goodbyes = []
with open('goodbyes.txt', encoding="utf-8") as file:
    for line in file:
        goodbye = line.rstrip()
        goodbyes.append(goodbye)
goodbyes

['до свидания',
 'до встречи',
 'прощайте',
 'счастливо',
 'пока',
 'всего доброго',
 'всего хорошего',
 'всего вам хорошего']

# Info parser

In [7]:

dialog_info = fact(
    'info', 
    ['greeting', 'goodbye', 'introduced', 'company_name']
)

GREETING = caseless_pipeline(greetings).interpretation(dialog_info.greeting) 
GOODBYE = caseless_pipeline(goodbyes).interpretation(dialog_info.goodbye) 

INTRODUSED = or_(
    rule(eq('меня'),  dictionary(names), eq('зовут')),
    rule(eq('меня'), eq('зовут'),  dictionary(names)),
    rule(eq('мое'), eq('имя'),  dictionary(names)),
    rule(eq('с'), eq('вами'), eq('говорит'),  dictionary(names))
).interpretation(dialog_info.introduced) 

company_start = eq('компания')
company_name = ttype('RU').interpretation(dialog_info.company_name)
COMPANY = rule(company_start, company_name)

INFO = or_(
    GREETING, GOODBYE, INTRODUSED, COMPANY
).interpretation(
    dialog_info
)

parser = Parser(INFO)

# Name parser

In [8]:
name_info = fact(
    'manager',
    ['name']
)

NAME = dictionary(names).interpretation(name_info.name).interpretation(name_info)
name_parser = Parser(NAME)

# Parsing

In [9]:
def parsing_facts(line):
    matches = list(parser.findall(line.lower()))
    if matches:
        facts = []
        for match in matches:
            fact = match.fact
            for f, pars in fact.as_json.items():
                facts.append('{0} - {1}'.format(f, pars))
                if fact.introduced is not None:
                    name = name_parser.find(fact.introduced).fact.name
                    facts.append('managers_name - {0}'.format(name))
        return '; '.join(facts)
    return ''

In [10]:
insight = pd.DataFrame({'insight':data[data.role == 'manager'].text.apply(parsing_facts)})

In [11]:
data_insights = pd.merge(data, insight, left_index=True, right_index=True, how='outer')
data_insights.insight.fillna('', inplace=True)

In [12]:
data_insights[['role', 'text', 'insight']]

,role,text,insight
0,client,Алло,
1,manager,Алло здравствуйте,greeting - здравствуйте
2,client,Добрый день,
3,manager,Меня зовут ангелина компания диджитал бизнес з...,introduced - меня зовут ангелина; managers_nam...
4,client,Ага,
...,...,...,...
475,manager,По поводу виджетов и с ними уже обсудите конкр...,
476,manager,Все я вам высылаю счет и с вами на связи если ...,
477,client,Спасибо спасибо,
478,client,Да да тогда созвонимся ага спасибо вам давайте,


In [13]:
dlg_id_info = data_insights[data_insights.insight != ''].groupby("dlg_id").agg({'insight': lambda x: '; '.join(x)})

In [14]:
for i in range(len(dlg_id_info)):
    print('dlg_id: ', i, '\n', \
          'требование - ', 'выполнено' if 'greeting' in dlg_id_info.insight[i].split() \
                                        and 'goodbye' in dlg_id_info.insight[i].split() else 'не выполнено', '\n\n',\
          '\n'.join(dlg_id_info.insight[i].split('; ')), '\n\n')
    

dlg_id:  0 
 требование -  выполнено 

 greeting - здравствуйте
introduced - меня зовут ангелина
managers_name - ангелина
company_name - диджитал
goodbye - всего хорошего
goodbye - до свидания 


dlg_id:  1 
 требование -  выполнено 

 greeting - здравствуйте
introduced - меня зовут ангелина
managers_name - ангелина
company_name - диджитал
goodbye - всего хорошего
goodbye - до свидания 


dlg_id:  2 
 требование -  не выполнено 

 greeting - здравствуйте
introduced - меня зовут ангелина
managers_name - ангелина
company_name - диджитал 


dlg_id:  3 
 требование -  выполнено 

 greeting - добрый день
introduced - меня максим зовут
managers_name - максим
company_name - китобизнес
company_name - хорошо
goodbye - всего доброго 


dlg_id:  4 
 требование -  не выполнено 

 goodbye - до свидания 


dlg_id:  5 
 требование -  не выполнено 

 goodbye - до свидания 


